In [1]:
from textwrap3 import wrap

text = """A Lion lay asleep in the forest, his great head resting on his paws. A timid little Mouse came upon him unexpectedly, and in her fright and haste to
get away, ran across the Lion's nose. Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.  "Spare me!" begged
the poor Mouse. "Please let me go and some day I will surely repay you."  The Lion was much amused to think that a Mouse could ever help him. But he
was generous and finally let the Mouse go.  Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's
net. Unable to free himself, he filled the forest with his angry roaring. The Mouse knew the voice and quickly found the Lion struggling in the net.
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.  "You laughed when I said I would repay
you," said the Mouse. "Now you see that even a Mouse can help a Lion." """
for wrp in wrap(text, 150):
  print (wrp)
print ("\n")

A Lion lay asleep in the forest, his great head resting on his paws. A timid little Mouse came upon him unexpectedly, and in her fright and haste to
get away, ran across the Lion's nose. Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.  "Spare me!" begged
the poor Mouse. "Please let me go and some day I will surely repay you."  The Lion was much amused to think that a Mouse could ever help him. But he
was generous and finally let the Mouse go.  Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's
net. Unable to free himself, he filled the forest with his angry roaring. The Mouse knew the voice and quickly found the Lion struggling in the net.
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.  "You laughed when I said I would repay
you," said the Mouse. "Now you see that even a Mouse can help a Lion."




In [2]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
summary_model = T5ForConditionalGeneration.from_pretrained('t5-base')
summary_tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary_model = summary_model.to(device)


KeyboardInterrupt: 

In [ ]:
import random
import numpy as np

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize

def postprocesstext (content):
  final=""
  for sent in sent_tokenize(content):
    sent = sent.capitalize()
    final = final +" "+sent
  return final


def summarizer(text,model,tokenizer):
  text = text.strip().replace("\n"," ")
  text = "summarize: "+text
  # print (text)
  max_len = 512
  encoding = tokenizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=3,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  min_length = 75,
                                  max_length=300)


  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]
  summary = dec[0]
  summary = postprocesstext(summary)
  summary= summary.strip()

  return summary


summarized_text = summarizer(text,summary_model,summary_tokenizer)


print ("\noriginal Text >>")
for wrp in wrap(text, 150):
  print (wrp)
print ("\n")
print ("Summarized Text >>")
for wrp in wrap(summarized_text, 150):
  print (wrp)
print ("\n")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



original Text >>
A Lion lay asleep in the forest, his great head resting on his paws. A timid little Mouse came upon him unexpectedly, and in her fright and haste to
get away, ran across the Lion's nose. Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.  "Spare me!" begged
the poor Mouse. "Please let me go and some day I will surely repay you."  The Lion was much amused to think that a Mouse could ever help him. But he
was generous and finally let the Mouse go.  Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's
net. Unable to free himself, he filled the forest with his angry roaring. The Mouse knew the voice and quickly found the Lion struggling in the net.
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.  "You laughed when I said I would repay
you," said the Mouse. "Now you see that even a Mouse can help a Lion."


Summarized Text >>

In [ ]:
total = 10

# **Answer Span Extraction (Keywords and Noun Phrases)**

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pke
import traceback


def get_nouns_multipartite(content):
    out=[]
    try:
        # extractor = spacy.load("en_core_web_sm")
        extractor = pke.unsupervised.MultipartiteRank()

        extractor.load_document(input=content,language='en')


        #    not contain punctuation marks or stopwords as candidates.
        pos = {'PROPN','NOUN'}
        #pos = {'PROPN','NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        # extractor.candidate_selection(pos=pos, stoplist=stoplist)
        extractor.candidate_selection(pos=pos)
        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=15)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from flashtext import KeywordProcessor

def get_keywords(originaltext,summarytext,total):
  keywords = get_nouns_multipartite(originaltext)
  print ("keywords unsummarized: ",keywords)
  keyword_processor = KeywordProcessor()
  for keyword in keywords:
    keyword_processor.add_keyword(keyword)

  keywords_found = keyword_processor.extract_keywords(summarytext)
  keywords_found = list(set(keywords_found))
  print ("keywords_found in summarized: ",keywords_found)

  important_keywords =[]
  for keyword in keywords:
    if keyword in keywords_found:
      important_keywords.append(keyword)

  return important_keywords[:total]


imp_keywords = get_keywords(text,summarized_text,total)
print (imp_keywords)


keywords unsummarized:  ['lion', 'mouse', 'forest', 'net', 'paws', 'day', 'hunter', 'toils', 'roaring', 'nose', 'voice', 'prey', 'nap', 'head', 'creature']
keywords_found in summarized:  ['mouse', 'prey', 'nose', 'lion', 'day', 'net', 'head', 'paws', 'forest']
['lion', 'mouse', 'forest', 'net', 'paws', 'day', 'nose', 'prey', 'head']


# **Question generation using T5**

In [ ]:
question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

In [ ]:
def get_question(context,answer,model,tokenizer):
  text = "context: {} answer: {}".format(context,answer)
  encoding = tokenizer.encode_plus(text,max_length=384, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)
  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


  dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question



for wrp in wrap(summarized_text, 150):
  print (wrp)
print ("\n")

for answer in imp_keywords:
  ques = get_question(summarized_text,answer,question_model,question_tokenizer)
  print (ques)
  print (answer.capitalize())
  print ("\n")


A lion lay asleep in the forest, his great head resting on his paws. The timid little mouse ran across the lion's nose and begged him to let him go.
"please let me go and some day i will surely repay you," said the mouse. He was generous and finally let the mouse go; some days later, while stalking
his prey, the lion was caught in an angry net.


What animal lay asleep in the forest?
Lion


Who begged the lion to let him go?
Mouse


Where did the lion lay asleep?
Forest


What was the lion caught in while stalking his prey?
Net


What was the lion's head resting on?
Paws


When did the mouse repay the lion?
Day


What part of the lion did the mouse run across to beg him to let him go?
Nose


What did the mouse stalk?
Prey


What part of the lion was resting on his paws?
Head




# **UI by using Gradio**

In [ ]:
import gradio as gr


context = gr.Textbox(lines=10, placeholder="Enter paragraph/content here...", label="Text")
total = gr.Slider(1,10, value=1,step=1, label="Total Number Of Questions")

output = gr.Markdown(  label="Question and Answers")


def generate_question(context,total):
  summary_text = summarizer(context,summary_model,summary_tokenizer)
  for wrp in wrap(summary_text, 150):
    print (wrp)
  np =  get_keywords(context,summary_text,total)
  print ("\n\nNoun phrases",np)
  output="<b style='color:black;'>Answer the following short questions.</b><br><br>"
  i=1
  for answer in np:
    ques = get_question(summary_text,answer,question_model,question_tokenizer)
    # output= output + ques + "\n" + "Ans: "+answer.capitalize() + "\n\n"
    output = output + "<b style='color:black;'>Q"+ str(i) + ") " + ques + "</b><br>"
    # output = output + "<br>"
    output = output + "<br>"
    i += 1
    
  output = output + "<br><b style='color:black;'>" + "Correct Answer Key:</b><br>"

  i=1
  for answer in np:
    output = output + "<b style='color:green;'>" + "Ans"+ str(i) + ": " +answer.capitalize()+  "</b>"
    output = output + "<br>"
    i += 1

  # summary ="Summary: "+ summary_text
  # for answer in np:
  #   summary = summary.replace(answer,"<b>"+answer+"</b>")
  #   summary = summary.replace(answer.capitalize(),"<b>"+answer.capitalize()+"</b>")
  # output = output + "<p>"+summary+"</p>"
  
  return output

iface = gr.Interface(
  fn=generate_question, 
  inputs=[context, total], 
  outputs=output, css=".gradio-container {background-image: url('file=blue.jpg')}", title="Generate Short Questions Within Seconds",
  allow_flagging="manual",flagging_options=["Save Data"])
  
# iface.launch(debug=True, share=True)

In [ ]:
import glob
import os.path
import pandas as pd

file =None

def filecreate(file,total):
    folder_path = r'C:\Users\HP\AppData\Local\Temp'
    file_type = r'\*txt'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

# import_file = pd.read_csv(max_file).replace('\n', "")
# print(import_file)

    fp = open(max_file, 'r')
    text = fp.read()
    print(text)
    print(fp)
    generated = generate_question(text, total)
    # return text
    return generated

filecreate(file,2)

A Lion lay asleep in the forest, his great head resting on his paws. A timid little Mouse came upon him unexpectedly, and in her fright and haste to
get away, ran across the Lion's nose. Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.  "Spare me!" begged
the poor Mouse. "Please let me go and some day I will surely repay you."  The Lion was much amused to think that a Mouse could ever help him. But he
was generous and finally let the Mouse go.  Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's
net. Unable to free himself, he filled the forest with his angry roaring. The Mouse knew the voice and quickly found the Lion struggling in the net.
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.  "You laughed when I said I would repay
you," said the Mouse. "Now you see that even a Mouse can help a Lion.
<_io.TextIOWrapper name='C:\\Users\\HP\

"<b style='color:black;'>Answer the following short question.</b><br><br><b style='color:black;'>Q1) What animal lay asleep in the forest?</b><br><br><b style='color:black;'>Q2) Who did the lion let go?</b><br><br><br><b style='color:black;'>Correct Answer Key:</b><br><b style='color:green;'>Ans1: Lion</b><br><b style='color:green;'>Ans2: Mouse</b><br>"

In [ ]:
import gradio as gr

context = gr.HTML(label="Text")
file = gr.File()
total = gr.Slider(1,10, value=1,step=1, label="Total Number Of Questions")


# output = gr.HTML( label="Question and Answers")

fface = gr.Interface(  
  fn=filecreate, 
  inputs=[file,total], 
  outputs=context, 
  css=".gradio-container {background-image: url('file=blue.jpg')}", title="Generate Fill-in-Blanks Within Seconds",
  allow_flagging="manual",flagging_options=["Save Data"])


# fface.launch(debug=True, share=True)



In [ ]:
demo = gr.TabbedInterface([iface, fface], ["Text", "Upload File"], css=".gradio-container {background-image: url('file=blue.jpg')}")
demo.launch(debug=True, share=True)

c:\Users\HP\anaconda3\lib\site-packages\gradio\blocks.py:726: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

Could not create share link, please check your internet connection.


A Lion lay asleep in the forest, his great head resting on his paws. A timid little Mouse came upon him unexpectedly, and in her fright and haste to
get away, ran across the Lion's nose. Roused from his nap, the Lion laid his huge paw angrily on the tiny creature to kill her.  "Spare me!" begged
the poor Mouse. "Please let me go and some day I will surely repay you."  The Lion was much amused to think that a Mouse could ever help him. But he
was generous and finally let the Mouse go.  Some days later, while stalking his prey in the forest, the Lion was caught in the toils of a hunter's
net. Unable to free himself, he filled the forest with his angry roaring. The Mouse knew the voice and quickly found the Lion struggling in the net.
Running to one of the great ropes that bound him, she gnawed it until it parted, and soon the Lion was free.  "You laughed when I said I would repay
you," said the Mouse. "Now you see that even a Mouse can help a Lion.
<_io.TextIOWrapper name='C:\\Users\\HP\